In [1]:
!pip install ftfy regex tqdm torch
!pip install git+https://github.com/openai/CLIP.git
!pip install pytorch-lightning

  Cloning https://github.com/openai/CLIP.git to /tmp/pip-req-build-vb_79s38
  Running command git clone -q https://github.com/openai/CLIP.git /tmp/pip-req-build-vb_79s38
  Resolved https://github.com/openai/CLIP.git to commit b46f5ac7587d2e1862f8b7b1573179d80dcdd620


In [2]:
import sys
sys.path.insert(0, '..')
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path+"/model")
import clip
import torch
from torch.utils.data import Dataset, DataLoader
import matplotlib.pyplot as plt
from PIL import Image
import pandas as pd
import numpy as np
import pytorch_lightning as pl
from pytorch_lightning import Trainer
from sklearn.metrics import top_k_accuracy_score
from sklearn.model_selection import train_test_split
import pickle
from tqdm import tqdm
import gc
import os
import pickle
from model.aliproduct_model import ALIPRODUCT_CLIP
from model.dataset import ALIPRODUCT_DATASET,prepare_data
from model.CONFIG import CONFIG
import faiss



In [3]:
clean_df = pd.read_csv("/home/ubuntu/Desktop/CVPR 2022 AliProducts Challenge/code/data/val_data_prompt_clean.csv")


In [4]:
# col_to_test = [c for c in clean_df.columns.values if "caption" in c ]
col_to_test = ["caption"]

In [5]:
col_to_test

['caption']

In [6]:
clip.available_models()

['RN50',
 'RN101',
 'RN50x4',
 'RN50x16',
 'RN50x64',
 'ViT-B/32',
 'ViT-B/16',
 'ViT-L/14']

In [7]:
model,preprocess = clip.load("ViT-L/14",CONFIG.device)

In [8]:
clip_model = ALIPRODUCT_CLIP(model)

In [9]:
def test_feature(caption_col):
    test_loader,df = prepare_data("/home/ubuntu/Desktop/CVPR 2022 AliProducts Challenge/code/data/val_data_prompt_clean.csv",
    CONFIG.test_image_data_dir,CONFIG.test_image_data_folder
    ,CONFIG.test_image_col,caption_col,CONFIG.test_batch_size,preprocess,CONFIG.global_random_state,test=True)
    trainer = Trainer(gpus=1)
    pred = trainer.predict(clip_model,test_loader)
    full_pred = tuple(map(torch.concat, zip(*pred)))
    image_embed,text_embed = full_pred
    faiss_index = faiss.IndexFlatIP(768)
    faiss_index_gpu = faiss.index_cpu_to_all_gpus(faiss_index)
    faiss_index_gpu.add(image_embed.numpy().astype(np.float32))
    top5_k_e,top5_k_y_pred = faiss_index_gpu.search(text_embed.numpy().astype(np.float32),5)
    top10_k_e,top10_k_y_pred = faiss_index_gpu.search(text_embed.numpy().astype(np.float32),10)
    top5_preds = np.array([1 if y_true in y_pred else 0 for y_true,y_pred in zip(df.index.values,top5_k_y_pred)])
    print(caption_col,"num correct pred",sum(top5_preds))
    top10_preds = np.array([1 if y_true in y_pred else 0 for y_true,y_pred in zip(df.index.values,top10_k_y_pred)])
    print(caption_col,"num correct pred",sum(top10_preds))

    top5_acc = top5_preds[top5_preds ==1].shape[0] / top5_preds.shape[0]
    top10_acc = top10_preds[top10_preds ==1].shape[0] / top10_preds.shape[0]
    return top5_acc,top10_acc

In [10]:
# def topk_accuracy(y_true, pred, label_map, k,threshold):
#     pred[pred < threshold] = 0
#     pred_count = []
#     remap = np.vectorize(lambda x: label_map[x]["label"])
#     #get top k values
#     top_k_pred = remap(pred.argsort(axis=1)[:, -k:][:, ::-1])
#     # append 1 if true label is in top k values else append 0
#     for i in range(len(y_true)):
#         if y_true[i] in top_k_pred[i]:
#             pred_count.append(1)
#         else:
#             pred_count.append(0)
#     return sum(pred_count)/len(pred_count)

In [11]:
col_name = []
top5_acc_preds = []
top10_acc_preds = []
for col in col_to_test:
    top5_acc,top10_acc = test_feature(col)
    col_name.append(col)
    top5_acc_preds.append(top5_acc)
    top10_acc_preds.append(top10_acc)
pred_df = {"caption_clean_method":col_name,"top_5":top5_acc_preds,"top_10":top10_acc_preds}

/home/ubuntu/anaconda3/lib/python3.9/site-packages/pytorch_lightning/loops/utilities.py:91: PossibleUserWarning: `max_epochs` was not set. Setting it to 1000 epochs. To train without an epoch limit, set `max_epochs=-1`.
  rank_zero_warn(
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2,3]


Predicting: 0it [00:00, ?it/s]

libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile


Predicting DataLoader 0:   3%|▎         | 3/98 [00:52<27:37, 17.44s/it]

libpng warning: iCCP: known incorrect sRGB profile


...

Predicting DataLoader 0:  69%|██████▉   | 68/98 [17:15<07:36, 15.22s/it]

libpng warning: iCCP: known incorrect sRGB profile


Predicting DataLoader 0:  73%|███████▎  | 72/98 [18:15<06:35, 15.22s/it]

libpng warning: iCCP: known incorrect sRGB profile


Predicting DataLoader 0: 100%|██████████| 98/98 [24:43<00:00, 15.14s/it]
caption num correct pred 5417
caption num correct pred 7726


In [12]:
pred_df = pd.DataFrame(pred_df)

In [13]:
clip_model = ALIPRODUCT_CLIP(model)
checkpoint = torch.load("../model_weights/ViT_large.pt")
for key in list(checkpoint["state_dict"].keys()):
    checkpoint["state_dict"][key.replace('module.', 'model.')] = checkpoint["state_dict"].pop(key)
clip_model.load_state_dict(checkpoint["state_dict"])

<All keys matched successfully>

In [14]:
top5_acc_trained_preds = []
top10_acc_trained_preds = []
for col in col_to_test:
    top5_acc,top10_acc = test_feature(col)
    top5_acc_trained_preds.append(top5_acc)
    top10_acc_trained_preds.append(top10_acc)
pred_df["top_5_trained"] = top5_acc_trained_preds
pred_df["top_10_trained"] = top10_acc_trained_preds

/home/ubuntu/anaconda3/lib/python3.9/site-packages/pytorch_lightning/loops/utilities.py:91: PossibleUserWarning: `max_epochs` was not set. Setting it to 1000 epochs. To train without an epoch limit, set `max_epochs=-1`.
  rank_zero_warn(
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2,3]


Predicting: 0it [00:00, ?it/s]

libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile


Predicting DataLoader 0:   0%|          | 0/98 [00:06<?, ?it/s]

libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile


...

Predicting DataLoader 0:  69%|██████▉   | 68/98 [17:17<07:37, 15.25s/it]

libpng warning: iCCP: known incorrect sRGB profile


Predicting DataLoader 0:  73%|███████▎  | 72/98 [18:18<06:36, 15.25s/it]

libpng warning: iCCP: known incorrect sRGB profile


Predicting DataLoader 0: 100%|██████████| 98/98 [24:46<00:00, 15.17s/it]
caption num correct pred 7769
caption num correct pred 11503


In [16]:
pred_df

,caption_clean_method,top_5,top_10,top_5_trained,top_10_trained
0,caption,0.10834,0.15452,0.15538,0.23006
